###**Desafío #3**

Dataset creado a partir de los últimos 16 discursos del presidente:
https://www.casarosada.gob.ar/informacion/discursos?start=1

El dataset es chico, para obtener mejores resultados se podrían agregar más discursos.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec

In [3]:
# carga del dataset

df = pd.read_csv('discursos.txt', sep='/n', header=None)

print("Cantidad de documentos:", df.shape[0])

df.head()

Cantidad de documentos: 373


<ipython-input-3-10847f988572>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('discursos.txt', sep='/n', header=None)


,0
0,Para nosotros la Argentina no es Puerto Madero...
1,Muchas veces se piensa que la obra pública se ...
2,Tenemos más de 6700 obras públicas iniciadas y...
3,Las empezamos y las terminamos y nunca a nadie...
4,"No, porque la verdad lo que hacemos es trabaja..."


In [4]:
# tokenización

sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [5]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [6]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=4,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [7]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [8]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 373


In [9]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 568


In [10]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=500,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 144229.984375
Loss after epoch 1: 78569.84375
Loss after epoch 2: 76993.234375
Loss after epoch 3: 77091.375
Loss after epoch 4: 76466.375
Loss after epoch 5: 75826.75
Loss after epoch 6: 76359.5
Loss after epoch 7: 77330.0
Loss after epoch 8: 78335.1875
Loss after epoch 9: 78209.8125
Loss after epoch 10: 77021.875
Loss after epoch 11: 76685.0
Loss after epoch 12: 73148.4375
Loss after epoch 13: 66092.75
Loss after epoch 14: 65302.875
Loss after epoch 15: 64040.375
Loss after epoch 16: 63876.0
Loss after epoch 17: 62348.375
Loss after epoch 18: 62094.75
Loss after epoch 19: 61057.75
Loss after epoch 20: 60141.375
Loss after epoch 21: 59357.0
Loss after epoch 22: 58588.5
Loss after epoch 23: 57900.5
Loss after epoch 24: 56794.75
Loss after epoch 25: 55235.25
Loss after epoch 26: 55842.25
Loss after epoch 27: 54654.75
Loss after epoch 28: 53802.125
Loss after epoch 29: 53729.5
Loss after epoch 30: 50305.75
Loss after epoch 31: 48066.5
Loss after epoch 32: 48095.75
Los

(3429947, 7340000)

In [11]:
#  10 palabras que MÁS se relacionan con crisis:
w2v_model.wv.most_similar(positive=["crisis"], topn=10)

[('registrado', 0.36815500259399414),
 ('existencia', 0.3645549416542053),
 ('guerra', 0.3602745831012726),
 ('seguro', 0.35744813084602356),
 ('allende', 0.3555392920970917),
 ('salvador', 0.34999924898147583),
 ('puntos', 0.3479922115802765),
 ('señor', 0.34477823972702026),
 ('económica', 0.3424667418003082),
 ('tuvieron', 0.3420993685722351)]

El nivel de similitud de todas las palabras gira en torno al 0.35%. Palabras que yo veo interesantes y que se relacionan con crisis son por ejemplo: "económica" (por la crisis economica que ocurre en el país), "guerra" (ya que el presidente hace referencia muchas veces a como afecta la guerra en la actual crisis).

In [13]:
# 20 palabras que MÁS se relacionan con crisis:
w2v_model.wv.most_similar(positive=["crisis"], topn=20)

[('registrado', 0.36815500259399414),
 ('existencia', 0.3645549416542053),
 ('guerra', 0.3602745831012726),
 ('seguro', 0.35744813084602356),
 ('allende', 0.3555392920970917),
 ('salvador', 0.34999924898147583),
 ('puntos', 0.3479922115802765),
 ('señor', 0.34477823972702026),
 ('económica', 0.3424667418003082),
 ('tuvieron', 0.3420993685722351),
 ('meses', 0.3391183912754059),
 ('ingreso', 0.33108747005462646),
 ('necesita', 0.33091825246810913),
 ('personas', 0.32273709774017334),
 ('única', 0.3218275308609009),
 ('chile', 0.3217339813709259),
 ('europa', 0.32145294547080994),
 ('climático', 0.32096269726753235),
 ('¿saben', 0.3205059766769409),
 ('seguridad', 0.31843745708465576)]

Cuando extiendo a las 20 palabras el análisis de similitud, encuentro interesantes y no muy lejos de las 10 anteriores: "ingreso", "chile", "europa" y "climático".

In [14]:
# 10 palabras que MÁS se relacionan con presindente:
w2v_model.wv.most_similar(positive=["presidente"], topn=10)

[('sr', 0.5867438316345215),
 ('medio', 0.41203129291534424),
 ('mí', 0.3969164788722992),
 ('camino', 0.37830936908721924),
 ('nada', 0.37771016359329224),
 ('da', 0.37216487526893616),
 ('10', 0.3706953525543213),
 ('atención', 0.36715447902679443),
 ('pobres', 0.3629620671272278),
 ('aires', 0.3599025309085846)]

Acá "sr" es la palabra más importante debido a que en muchos discursos les habla o presenta a otros presidentes y en la transcripccion aparece: "sr. presidente de ...".

In [17]:
# 10 palabras que MÁS se relacionan con argentina:
w2v_model.wv.most_similar(positive=["argentina"], topn=10)

[('grande', 0.31717249751091003),
 ('poderosos', 0.305957168340683),
 ('vivir', 0.28838127851486206),
 ('espero', 0.28388407826423645),
 ('justicia', 0.2838115692138672),
 ('europa', 0.2836097776889801),
 ('ingreso', 0.28335025906562805),
 ('norte', 0.2787700593471527),
 ('cerca', 0.2728211581707001),
 ('hubiera', 0.26647135615348816)]

Entiendo que la palabra "grande" se relaciona fuertemente con la palabra "argentina" ya que el presiente utiliza muchas veces en su discurso frases como "la argentina grande" o "por una argentina grande"

In [18]:
# 10 palabras que MÁS se relacionan con inflación:
w2v_model.wv.most_similar(positive=["inflación"], topn=10)

[('dejaron', 0.5794287323951721),
 ('54', 0.5729076862335205),
 ('deuda', 0.45120707154273987),
 ('puntos', 0.44852834939956665),
 ('europa', 0.4336605370044708),
 ('jóvenes', 0.39196324348449707),
 ('poderosos', 0.3888978660106659),
 ('guerra', 0.374417245388031),
 ('50', 0.3675536811351776),
 ('ingresos', 0.36685773730278015)]

Es interesante acá algunas palabras relacionadas con inflación como ser: "dejaron" (ya que el presidente muchas veces hacer referencia a la inflación que le dejó el gobierno anterior). Tambien interesante ver "deuda" y las palabras relacionadas con crisis como "guerra" y "europa".

In [19]:
# 10 palabras que MÁS se relacionan con deuda:
w2v_model.wv.most_similar(positive=["deuda"], topn=10)

[('54', 0.5363849997520447),
 ('dejaron', 0.5341254472732544),
 ('inflación', 0.45120713114738464),
 ('brics', 0.3962947726249695),
 ('guerra', 0.3710784912109375),
 ('orden', 0.3643539547920227),
 ('50', 0.3598240315914154),
 ('sequía', 0.35774698853492737),
 ('terminar', 0.3514992594718933),
 ('cuesta', 0.3449435234069824)]

Acá palabras que me parecen intereante relacionadas con "deuda" son "dejaron", "inflación", nuevamente "guerra" y "sequía". "54" aparece dado que muchas veces hace referencia en sus discursos a "54 puntos de inflación".

In [21]:
# palabras que MÁS se relacionan con pobres:
w2v_model.wv.most_similar(positive=["pobres"], topn=10)

[('morir', 0.5794013142585754),
 ('cuántos', 0.4934534430503845),
 ('jóvenes', 0.48916947841644287),
 ('podían', 0.44801467657089233),
 ('podía', 0.41555091738700867),
 ('madero', 0.4046913981437683),
 ('esfuerzo', 0.4032368063926697),
 ('cobrar', 0.39914682507514954),
 ('hubiera', 0.39887991547584534),
 ('camiones', 0.39360371232032776)]

Acá "morir" tiene que ver con frases relacionadas con la pandimia ... algo así como "puenden morir tanto pobres como ricos". Interesante que aparezca "madero" en relación con pobres ya que es una referencia al departamento donde vivia en "puerto madero", lo mismo pasa con "camiones" y "madero" ya que hace referencia a las obras que evitan el paso de camiones por dicho barrio.

In [22]:
# 10 palabras que MÁS se relacionan con gracias:
w2v_model.wv.most_similar(positive=["gracias"], topn=10)

[('buenas', 0.6161782741546631),
 ('amigos', 0.49152833223342896),
 ('muchas', 0.4729737639427185),
 ('salvador', 0.4394723176956177),
 ('sr', 0.4391665756702423),
 ('provincias', 0.4319251477718353),
 ('logramos', 0.42273595929145813),
 ('prepaga', 0.4053828716278076),
 ('enorme', 0.40488582849502563),
 ('catamarca', 0.4036981165409088)]

Acá "buenas", "muchas", "sr" y "enorme" son paralabras que se encuentran en los saludos finales del presidente y seguramente eso genera la relación con "gracias"

In [23]:
# 10 palabras que MÁS se relacionan con madero:
w2v_model.wv.most_similar(positive=["madero"], topn=10)

[('puerto', 0.7250540852546692),
 ('camiones', 0.5064966082572937),
 ('señor', 0.48349839448928833),
 ('vecinos', 0.4253254532814026),
 ('pobres', 0.4046913981437683),
 ('vivir', 0.4035724103450775),
 ('mía', 0.39223095774650574),
 ('bajo', 0.38422292470932007),
 ('morir', 0.3832818865776062),
 ('cuántos', 0.3694327175617218)]

Acá "puerto" es algo lógico al igual que "camiones" y "vecinos".

In [25]:
# 10 palabras que MENOS se relacionan con presidente:
w2v_model.wv.most_similar(negative=["presidente"], topn=10)

[('política', 0.03661388158798218),
 ('nuestros', -0.009348240680992603),
 ('vamos', -0.010918217711150646),
 ('casas', -0.016569530591368675),
 ('está', -0.016969837248325348),
 ('viven', -0.020670242607593536),
 ('pasa', -0.023038819432258606),
 ('todo', -0.030221564695239067),
 ('necesitan', -0.030956977978348732),
 ('acueducto', -0.03388890624046326)]

Bueno acá resulta gracioso que una de las palabras sea "política" .

In [21]:
# 10 palabras que MENOS se relacionan con democracia:
w2v_model.wv.most_similar(negative=["democracia"], topn=10)

[('agua', 0.03346685320138931),
 ('sin', -0.002541245659813285),
 ('estado', -0.003481603693217039),
 ('rosa', -0.003740126034244895),
 ('este', -0.004348464775830507),
 ('ir', -0.004954023286700249),
 ('toda', -0.009563903324306011),
 ('ustedes', -0.017769258469343185),
 ('en', -0.02312721684575081),
 ('río', -0.02974279783666134)]

Al igual que en el anterior, que aparezca "estado" en la lista es interesante.

In [27]:
 # pruebo el método `get_vector`
vector_pais = w2v_model.wv.get_vector("país")
print(vector_pais)

[ 5.27771050e-03  1.13424055e-01 -2.13760007e-02  2.92052329e-01
  2.34538510e-01 -2.66555101e-01 -1.60463035e-01  3.66843671e-01
 -5.21568894e-01  2.10860565e-01 -1.44894898e-01  4.16690588e-01
 -1.23444356e-01 -2.63701528e-01 -5.87549359e-02 -2.91965436e-02
  2.44345114e-01 -6.58347830e-02 -5.12017757e-02  6.11103140e-02
  2.06846774e-01 -3.82328898e-01 -2.76094049e-01 -3.44769657e-01
 -3.77408922e-01 -8.02442431e-02 -1.20792978e-01  7.80528933e-02
 -2.66833603e-01  4.56329621e-02  3.53437454e-01 -1.25087807e-02
  2.18899846e-01 -1.66512027e-01 -2.08525255e-01  1.69948518e-01
  6.48531735e-01 -1.90185502e-01 -1.87197149e-01  3.32342267e-01
 -3.42956036e-01 -1.15144804e-01  2.93077081e-01 -3.62102985e-01
 -2.61304945e-01  3.63150216e-03  1.78271085e-01 -3.13468456e-01
  2.20999092e-01  2.92031705e-01  5.65231979e-01  1.56325579e-01
 -1.49731532e-01 -1.37633964e-01 -7.45922253e-02  5.20622611e-01
  3.38189244e-01 -8.32374170e-02  4.61579889e-01  1.12510219e-01
  5.37996650e-01 -1.72462

In [28]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_pais)

[('país', 1.0),
 ('grande', 0.35937637090682983),
 ('cuatro', 0.3562638461589813),
 ('argentinas', 0.3510570228099823),
 ('chicos', 0.3400222957134247),
 ('públicas', 0.33632951974868774),
 ('norte', 0.33550217747688293),
 ('energía', 0.32807162404060364),
 ('llevando', 0.3116864264011383),
 ('humano', 0.3051295876502991)]

In [29]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [30]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [31]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [98]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)